In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from webdriver_manager.chrome import ChromeDriverManager



In [2]:
#declare urls
url1 = 'https://redplanetscience.com/'
url2 = 'https://spaceimages-mars.com/'
url3 = 'https://galaxyfacts-mars.com/'
url4 = 'https://marshemispheres.com/'


In [32]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\soche\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


In [4]:
#scrape news
browser.visit(url1)
html = browser.html
soup = bs(html, 'html.parser')

news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb
Along with capturing an image before its steepest ascent ever, the robotic explorer filmed its "selfie stick," or robotic arm, in action.


In [5]:
#scrape featured image
browser.visit(url2)
html = browser.html
soup = bs(html, 'html.parser')

featured_image_url = url2 + soup.find('img', class_='headerimage')['src']

print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


In [6]:
#scrape facts in pandas
tables = pd.read_html(url3)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [26]:
#select table for mars facts
mars_facts_df = tables[0]


In [31]:
#rename columns
renamed_mars_facts_df = mars_facts_df.rename(columns={0:'Description', 1:'Mars', 2:'Earth'})

#render html table from pandas
mars_facts_table = renamed_mars_facts_df.to_html(index=False)

#clean 
final_mars_facts_table = mars_facts_table.replace('\n', '')
final_mars_facts_table


'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Description</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <td>Mars - Earth Comparison</td>      <td>Mars</td>      <td>Earth</td>    </tr>    <tr>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <td>Temperature:</td>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [60]:
#scrape 4 hemispheres
browser.visit(url4)
html = browser.html
soup = bs(html, 'html.parser')

#get 4 hemispheres
hemispheres = soup.find_all('div', class_='description')
#hemispheres_link = hemispheres.find_all('a')
#print (hemispheres)

#set up hemi url list
hemisphere_image_urls = []

#loop through each hemisphere div
for hemi in hemispheres:
    hemi_dict = {}
    
    #extract hemisphere title
    hemi_title = hemi.find('h3').text
        
    #go to the link in div
    browser.links.find_by_partial_text(hemi_title).click()
    hemi_html = browser.html
    hemi_soup = bs(hemi_html, 'html.parser')
    
    #pull href to complete image urls
    temp_div = hemi_soup.find('div', class_='downloads')
    hemi_image = temp_div.find('a')['href']
    hemi_image_link = url4 + hemi_image
    #print(temp_div)
    
    #add values to dictionary
    hemi_dict['title'] = hemi_title
    hemi_dict['img_url'] = hemi_image_link
    
    #add dictionary to list hemisphere_image_urls
    hemisphere_image_urls.append(hemi_dict)
    
    #go back a page
    browser.back()

In [61]:
#display list
print (hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]
